In [1]:
import json
import torch
import commentjson
import numpy as np
import os
import gc
from transformers import EarlyStoppingCallback
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    DataCollatorForTokenClassification
)
from datasets import Dataset, Features, Value, ClassLabel, Sequence
from sklearn.metrics import accuracy_score
from seqeval.metrics import f1_score, classification_report
import logging
from sklearn.model_selection import train_test_split

# --- 0. 기본 설정 ---
MODEL_NAME = "klue/roberta-base"
MAX_LEN = 128
EPOCHS = 5
BATCH_SIZE = 8

# 모델 저장 경로 설정
INTENT_MODEL_DIR = "./models/intent"
NER_MODEL_DIR = "./models/ner"
INTENT_LABEL_PATH = os.path.join(INTENT_MODEL_DIR, "intent_labels.jsonc")
NER_LABEL_PATH = os.path.join(NER_MODEL_DIR, "ner_labels.jsonc")

# --- 1. 데이터 로드 ---
# 1.1 Intent 분류를 위한 데이터
def load_intent_data():
    with open('intent_label_list.jsonc', 'r', encoding='utf-8') as f:
        intent_label_list = commentjson.load(f)

    intent_label_to_id = {label: i for i, label in enumerate(intent_label_list)}
    intent_id_to_label = {i: label for label, i in intent_label_to_id.items()}

    with open('intent_data.jsonc', 'r', encoding='utf-8') as f:
        intent_data = commentjson.load(f)

    return intent_data, intent_label_list, intent_label_to_id, intent_id_to_label

# 1.2 NER을 위한 데이터
def load_ner_data():
    with open('ner_data.jsonc', 'r', encoding='utf-8') as f:
        loaded_ner_data = commentjson.load(f)

    ner_data = []
    for item in loaded_ner_data:
        entities_as_tuples = [tuple(entity_list) for entity_list in item.get("entities", [])]
        ner_data.append({"text": item.get("text", ""), "entities": entities_as_tuples})

    return ner_data

# --- 2. Intent 분류 모델 훈련 ---
def train_intent_model():
    print("\n" + "="*50)
    print("Intent 분류 모델 훈련 시작")
    print("="*50)

    intent_data, intent_label_list, intent_label_to_id, intent_id_to_label = load_intent_data()

    num_intent_labels = len(intent_label_list)
    print(f"Intent 레이블 ({num_intent_labels}개) 사용: {intent_label_to_id}")

    intent_features = Features({
        'text': Value('string'),
        'label': ClassLabel(num_classes=num_intent_labels, names=intent_label_list)
    })

    intent_dataset = Dataset.from_list(intent_data, features=intent_features)

    # 학습/평가 데이터 분리
    train_test_datasets = intent_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_datasets["train"]
    eval_dataset = train_test_datasets["test"]

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    def preprocess_intent_data(examples):
        tokenized = tokenizer(
            examples['text'],
            truncation=True,
            max_length=MAX_LEN
        )
        return {
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"]
        }

    tokenized_train_dataset = train_dataset.map(
        preprocess_intent_data,
        batched=True,
        remove_columns=['text']
    )

    tokenized_eval_dataset = eval_dataset.map(
        preprocess_intent_data,
        batched=True,
        remove_columns=['text']
    )

    print(f"Intent 데이터 샘플: {tokenized_train_dataset[0]}")

    intent_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    intent_model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_intent_labels,
        id2label=intent_id_to_label,
        label2id=intent_label_to_id
    )

    def compute_intent_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {"accuracy": accuracy_score(labels, predictions)}

    training_args = TrainingArguments(
        output_dir="./results/intent",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        logging_dir='./logs/intent',
        logging_steps=10,
        load_best_model_at_end=True,
        eval_steps=100,
        eval_strategy="steps",
        save_steps=100,
        save_total_limit=2,
        report_to="none",
        learning_rate=5e-5,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
    )

    intent_trainer = Trainer(
        model=intent_model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=intent_data_collator,
        compute_metrics=compute_intent_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    intent_trainer.train()
    eval_result = intent_trainer.evaluate()
    print(f"Intent 모델 평가 결과: {eval_result}")
    print("Intent 모델 훈련 완료")

    # 모델 및 토크나이저 저장
    os.makedirs(INTENT_MODEL_DIR, exist_ok=True)
    intent_model.save_pretrained(INTENT_MODEL_DIR, safe_serialization=False)
    tokenizer.save_pretrained(INTENT_MODEL_DIR)

    # 레이블 정보 저장
    with open(INTENT_LABEL_PATH, 'w', encoding='utf-8') as f:
        json.dump({
            "id2label": intent_id_to_label,
            "label2id": intent_label_to_id
        }, f, ensure_ascii=False, indent=2)

    print(f"Intent 모델 및 레이블 정보 저장 완료: {INTENT_MODEL_DIR}")

# --- 3. NER 모델 훈련 ---
def train_ner_model():
    print("\n" + "="*50)
    print("NER 모델 훈련 시작")
    print("="*50)

    ner_data = load_ner_data()

    # 3.1 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 디버깅: 토크나이저 테스트
    test_text = "인문학"
    test_tokens = tokenizer.tokenize(test_text)
    print(f"토크나이저 테스트 '{test_text}' -> {test_tokens}")

    # 3.2 NER 레이블 정의 (BIO 형식)
    # 먼저 모든 엔터티 타입 추출
    entity_types = sorted(list(set(label for item in ner_data for _, _, label in item["entities"])))
    print(f"찾은 엔터티 타입: {entity_types}")

    ner_labels = ["O"]  # Outside tag
    for entity_type in entity_types:
        ner_labels.extend([f"B-{entity_type}", f"I-{entity_type}"])

    ner_label2id = {label: i for i, label in enumerate(ner_labels)}
    ner_id2label = {i: label for label, i in ner_label2id.items()}

    print(f"NER 레이블 ({len(ner_labels)}개): {ner_labels}")

    # 3.3 NER 데이터 전처리 - 문자 단위 접근
    preprocessed_ner_data = []

    print("\n--- NER 데이터 전처리 시작 ---")

    for example_idx, example in enumerate(ner_data):
        text = example["text"]
        entities = example["entities"]

        if example_idx < 3:  # 처음 몇 개 예시만 상세 출력
            print(f"\n[데이터 {example_idx}] 텍스트: \"{text}\"")
            print(f"  정의된 엔티티: {entities}")

        # 1. 문자 단위 BIO 태깅 초기화
        char_labels = ["O"] * len(text)

        # 2. 엔티티에 따라 BIO 태그 할당
        for start_char, end_char, entity_type in entities:
            # 범위 체크 및 수정
            if start_char < 0:
                start_char = 0
            if end_char > len(text):
                end_char = len(text)

            if start_char < end_char and start_char < len(text):
                for i in range(start_char, end_char):
                    if i == start_char:
                        char_labels[i] = f"B-{entity_type}"
                    else:
                        char_labels[i] = f"I-{entity_type}"

        if example_idx < 3 or start_char >= len(text) or end_char > len(text) or start_char < 0:
            print(f"  문제가 있는 엔티티: ({start_char}, {end_char}, {entity_type})")
            print(f"  텍스트 길이: {len(text)}")

        if example_idx < 3:  # 상세 디버깅 출력
            print(f"  문자별 BIO 태그:")
            for i, (char, label) in enumerate(zip(text, char_labels)):
                print(f"    '{char}': {label}")

        # 3. 토큰화 및 토큰-문자 정렬
        tokenized = tokenizer(text, return_offsets_mapping=True, add_special_tokens=True)
        tokens = tokenizer.convert_ids_to_tokens(tokenized['input_ids'])
        offset_mapping = tokenized['offset_mapping']

        if example_idx < 3:  # 상세 디버깅 출력
            print(f"  토큰화 결과: {tokens}")
            print(f"  오프셋 매핑: {offset_mapping}")

        # 4. 토큰별 레이블 할당
        token_labels = []
        for i, (start, end) in enumerate(offset_mapping):
            # 특수 토큰 처리
            if start == end:
                token_label = -100  # ignore_index
            else:
                # 토큰 시작 위치의 문자 레이블 사용
                char_label = char_labels[start]
                token_label = ner_label2id[char_label]

                # 서브워드 토큰은 I- 태그로 변환 (WordPiece ##로 시작하는 경우)
                if i > 0 and tokens[i].startswith("##"):
                    prev_label = token_labels[-1]
                    if prev_label != -100 and ner_id2label[prev_label].startswith("B-"):
                        # B- -> I- 변환
                        entity_type = ner_id2label[prev_label][2:]  # "B-genre" -> "genre"
                        token_label = ner_label2id[f"I-{entity_type}"]

            token_labels.append(token_label)

        if example_idx < 3:  # 상세 디버깅 출력
            print(f"  최종 토큰 레이블: {[ner_id2label.get(l, 'IGN') for l in token_labels]}")

        # 5. 레이블 ID로 변환
        preprocessed_ner_data.append({
            "text": text,
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"],
            "labels": token_labels
        })

    # 3.4 데이터셋 생성
    ner_features = Features({
        'text': Value('string'),
        'input_ids': Sequence(Value('int32')),
        'attention_mask': Sequence(Value('int32')),
        'labels': Sequence(Value('int32'))
    })

    ner_dataset = Dataset.from_list(preprocessed_ner_data, features=ner_features)

    # 학습/평가 데이터셋 분리
    train_test_datasets = ner_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_datasets["train"]
    eval_dataset = train_test_datasets["test"]

    print(f"NER 훈련 데이터 크기: {len(train_dataset)}")
    print(f"NER 평가 데이터 크기: {len(eval_dataset)}")
    print(f"NER 데이터 샘플: {train_dataset[0]}")

    # 3.5 데이터 콜레이터 설정
    ner_data_collator = DataCollatorForTokenClassification(
        tokenizer=tokenizer,
        padding=True,
        max_length=MAX_LEN,
        pad_to_multiple_of=8
    )

    # 3.6 모델 로드
    ner_model = AutoModelForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(ner_labels),
        id2label=ner_id2label,
        label2id=ner_label2id
    )

    # 3.7 평가 지표 계산 함수
    def compute_ner_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        # 실제 토큰의 예측값과 레이블만 추출
        true_predictions = []
        true_labels = []

        for prediction, label in zip(predictions, labels):
            true_pred = []
            true_label = []

            for p, l in zip(prediction, label):
                if l != -100:  # -100은 무시
                    true_pred.append(ner_id2label[p])
                    true_label.append(ner_id2label[l])

            true_predictions.append(true_pred)
            true_labels.append(true_label)

        # seqeval의 f1_score 계산
        try:
            f1 = f1_score(true_labels, true_predictions)
            report = classification_report(true_labels, true_predictions, digits=4)
            print("\nNER Classification Report:\n", report)

            # 세부 클래스별 결과 분석
            class_results = {}
            for label in ner_labels:
                if label != "O" and label.startswith("B-"):
                    entity_type = label[2:]  # "B-genre" -> "genre"
                    label_f1 = f1_score([[label]], [[label]], average='macro')
                    class_results[entity_type] = label_f1

            # 상세 예측 예시 출력
            print("\n예측 예시:")
            for i in range(min(5, len(true_labels))):
                print(f"실제: {true_labels[i]}")
                print(f"예측: {true_predictions[i]}")
                print("---")

            return {
                "f1": f1,
                **{f"f1_{key}": val for key, val in class_results.items()}
            }
        except Exception as e:
            print(f"Error calculating NER metrics: {e}")
            return {"f1": 0.0}

    # 3.8 훈련 설정
    training_args = TrainingArguments(
        output_dir="./results/ner",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        logging_dir='./logs/ner',
        logging_steps=10,
        save_steps=100,
        eval_steps=100,
        eval_strategy="steps",
        load_best_model_at_end=True,
        save_total_limit=2,
        greater_is_better=True,
        metric_for_best_model="f1",
        weight_decay=0.01,
        report_to="none",
        learning_rate=5e-5,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",

    )

    # 3.9 Trainer 정의
    ner_trainer = Trainer(
        model=ner_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=ner_data_collator,
        compute_metrics=compute_ner_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # 3.10 모델 훈련
    ner_trainer.train()
    eval_result = ner_trainer.evaluate()
    print(f"NER 모델 평가 결과: {eval_result}")
    print("NER 모델 훈련 완료")

    # 3.11 모델 및 레이블 정보 저장
    os.makedirs(NER_MODEL_DIR, exist_ok=True)
    gc.collect()
    ner_model.cpu()
    ner_model.save_pretrained(NER_MODEL_DIR, safe_serialization=False)
    tokenizer.save_pretrained(NER_MODEL_DIR)

    # 레이블 매핑 저장
    with open(NER_LABEL_PATH, 'w', encoding='utf-8') as f:
        json.dump({
            "id2label": ner_id2label,
            "label2id": ner_label2id
        }, f, ensure_ascii=False, indent=2)

    print(f"NER 모델 및 레이블 정보 저장 완료: {NER_MODEL_DIR}")


# --- 메인 함수 ---
def main():
    print("\n" + "="*50)
    print("도서 검색 NLU 모델 훈련 시작")
    print("="*50)

    # Intent 모델 훈련
    train_intent_model()

    # NER 모델 훈련
    train_ner_model()

    print("\n" + "="*50)
    print("도서 검색 NLU 모델 훈련 완료")
    print("="*50)


if __name__ == "__main__":
    main()



도서 검색 NLU 모델 훈련 시작

Intent 분류 모델 훈련 시작
Intent 레이블 (40개) 사용: {'search_book_title': 0, 'search_book_author': 1, 'search_book_location': 2, 'check_book_availability': 3, 'get_bestseller': 4, 'get_new_releases': 5, 'request_recommendation_genre': 6, 'request_recommendation_mood': 7, 'request_recommendation_topic': 8, 'request_recommendation_similar': 9, 'request_recommendation_reader': 10, 'search_space_availability': 11, 'reserve_space': 12, 'get_space_info': 13, 'check_space_reservation': 14, 'cancel_space_reservation': 15, 'search_program': 16, 'apply_program': 17, 'get_program_info': 18, 'check_program_application': 19, 'cancel_program_application': 20, 'get_library_info': 21, 'inquire_service': 22, 'manage_membership': 23, 'check_loan_status': 24, 'extend_loan': 25, 'reserve_book': 26, 'check_reservation_status': 27, 'cancel_book_reservation': 28, 'check_overdue_status': 29, 'report_lost_item': 30, 'greeting': 31, 'gratitude': 32, 'closing': 33, 'affirmative': 34, 'negative': 35, 'a

Map:   0%|          | 0/708 [00:00<?, ? examples/s]

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

Intent 데이터 샘플: {'label': 2, 'input_ids': [0, 15382, 1459, 8075, 4318, 1556, 3915, 5527, 2343, 2388, 7187, 18, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(



NER Classification Report:
                    precision    recall  f1-score   support

           author     0.9730    0.9908    0.9818       109
      call_number     0.0000    0.0000    0.0000         0
         category     0.0000    0.0000    0.0000         1
         duration     0.0000    0.0000    0.0000         1
       event_type     0.5000    0.3333    0.4000         3
           format     0.9697    0.9846    0.9771       130
             isbn     0.0000    0.0000    0.0000         1
library_info_type     0.0000    0.0000    0.0000         1
         location     0.1667    1.0000    0.2857         1
        lost_item     0.0000    0.0000    0.0000         1
             mood     0.0000    0.0000    0.0000         2
       num_people     0.0000    0.0000    0.0000         0
            title     0.9582    0.9703    0.9642       236
            topic     0.0000    0.0000    0.0000         1

        micro avg     0.9492    0.9589    0.9540       487
        macro avg     0.2

c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\spring\CaterpillarAi\venv\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`

c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



NER Classification Report:
                    precision    recall  f1-score   support

           author     0.9730    0.9908    0.9818       109
      call_number     0.0000    0.0000    0.0000         0
         category     0.0000    0.0000    0.0000         1
         duration     0.0000    0.0000    0.0000         1
       event_type     0.5000    0.3333    0.4000         3
           format     0.9697    0.9846    0.9771       130
             isbn     0.0000    0.0000    0.0000         1
library_info_type     0.0000    0.0000    0.0000         1
         location     0.1667    1.0000    0.2857         1
        lost_item     0.0000    0.0000    0.0000         1
             mood     0.0000    0.0000    0.0000         2
       num_people     0.0000    0.0000    0.0000         0
            title     0.9582    0.9703    0.9642       236
            topic     0.0000    0.0000    0.0000         1

        micro avg     0.9492    0.9589    0.9540       487
        macro avg     0.2